## Uncertainty Quantification
For the following tasks use the GEFCOM dataset (data columns: YYYYMMDD, HH, zonal price, system load, zonal load, day-of-the-week)

In [68]:
import polars as pl
import numpy as np
from scipy.stats import norm, chi2
from sklearn.linear_model import LinearRegression


In [69]:
df = pl.read_csv('data/gefcom.txt', separator='\t', has_header=True)
df = df.with_columns(
    pl.col("Hour").cast(pl.UInt8),
    pl.col("Weekday").cast(pl.UInt8),
).drop(pl.col(""))
df = df.to_dummies("Weekday") 
df

Date,Hour,Price,SystemLoad,Load,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Weekday_7
f64,u8,f64,f64,f64,u8,u8,u8,u8,u8,u8,u8
2.0110101e7,0,43.17,15187.0,5091.0,0,0,0,0,0,1,0
2.0110101e7,1,36.24,14464.0,4918.0,0,0,0,0,0,1,0
2.0110101e7,2,34.64,13940.0,4763.0,0,0,0,0,0,1,0
2.0110101e7,3,33.76,13609.0,4660.0,0,0,0,0,0,1,0
2.0110101e7,4,33.08,13391.0,4599.0,0,0,0,0,0,1,0
…,…,…,…,…,…,…,…,…,…,…,…
2.0131217e7,19,113.92,23091.0,7167.0,0,1,0,0,0,0,0
2.0131217e7,20,107.26,22504.0,6958.0,0,1,0,0,0,0,0
2.0131217e7,21,89.02,21538.0,6707.0,0,1,0,0,0,0,0


First we will obtain point forecasts from the following naive model:   
$$ \hat{P}_{d,h} = (P_{d-1,h} + P_{d-7,h})/2 $$

Next, we will calculate forecasts for both left-tail and right-tail quantiles, e.g. 5% and 95% quantiles using:
- historical simulation
- conformal prediction
- normal distribution (estimate the standard deviation)

We will calibrate the parameters for each hour separately, using a 364-day calibration window. For each quantile level we will calculate:
- coverage
- pinball loss
- p-value from the Kupiec test

In [70]:
df = df.with_columns(
    pl.col("Price").shift(24).alias("PriceLag1"),
    pl.col("Price").shift(168).alias("PriceLag7"),
).drop_nulls()

df = df.with_columns(
    (pl.col("PriceLag1")/2 + pl.col("PriceLag7")/2).alias("Naive")
)

df

Date,Hour,Price,SystemLoad,Load,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Weekday_7,PriceLag1,PriceLag7,Naive
f64,u8,f64,f64,f64,u8,u8,u8,u8,u8,u8,u8,f64,f64,f64
2.0110108e7,0,49.22,16269.0,5335.0,0,0,0,0,0,1,0,45.53,43.17,44.35
2.0110108e7,1,42.69,15568.0,5099.0,0,0,0,0,0,1,0,41.53,36.24,38.885
2.0110108e7,2,40.33,15160.0,4939.0,0,0,0,0,0,1,0,40.54,34.64,37.59
2.0110108e7,3,39.81,14912.0,4840.0,0,0,0,0,0,1,0,37.76,33.76,35.76
2.0110108e7,4,39.38,14783.0,4759.0,0,0,0,0,0,1,0,38.28,33.08,35.68
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2.0131217e7,19,113.92,23091.0,7167.0,0,1,0,0,0,0,0,138.15,63.34,100.745
2.0131217e7,20,107.26,22504.0,6958.0,0,1,0,0,0,0,0,135.53,57.0,96.265
2.0131217e7,21,89.02,21538.0,6707.0,0,1,0,0,0,0,0,110.64,53.52,82.08


In [ ]:
def hs(forecasts, obs, alpha):
    """
    Calculate prediction interval using historical simulation

    Args:
        forecasts (1d numpy array): forecasts of the timeseries
        obs (1d numpy array): observed values of the timeseries to forecast
        alpha (float): describes the confidence of the prediciton interval

    Returns:
        2-tuple of floats: lower and upper bound of the alpha prediction interval
    """
    errors = forecasts - obs
    ql, qh = np.quantile(errors, [(1-alpha)/2, 1-(1-alpha)/2])
    return (ql, qh)

def cp(forecasts, obs, alpha):
    """
    Calculate prediction interval using conformal prediction

    Args:
        forecasts (1d numpy array): forecasts of the timeseries
        obs (1d numpy array): observed values of the timeseries to forecast
        alpha (float): describes the confidence of the prediciton interval

    Returns:
        2-tuple of floats: lower and upper bound of the alpha prediction interval
    """
    errors = np.abs(forecasts - obs)
    q = np.quantile(errors, alpha)
    return (-q, q)

def normal(forecasts, obs, alpha):
    """
    Calculate prediction interval using normal error distribution

    Args:
        forecasts (1d numpy array): forecasts of the timeseries
        obs (1d numpy array): observed values of the timeseries to forecast
        alpha (float): describes the confidence of the prediciton interval

    Returns:
        2-tuple of floats: lower and upper bound of the alpha prediction interval
    """
    errors = forecasts - obs
    sigma = np.std(errors)
    q = norm.ppf(1-(1-alpha)/2, loc=0, scale=sigma)
    return (-q, q)

# the above functions return the prediction intervals which are not yet centered around the point forecast
# to build a prediciton interval for forecast `y_hat`, you have to add `y_hat` to the lower and upper bound of the output of `hs` or `cp`

In [72]:
method = cp    # select cp, hs or normal
window = 364   # number of observations and forecasts used for estimating quantile predictions
alpha = 0.9    # confidence level of the prediction interval, i.e., if we want the 5% and 95% quantile, we select a 90% prediction interval
quantile_forecasts = np.full((len(df)//24, 24, 2), np.nan)
for h in range(0, 24):
    df_hour = df.filter(pl.col("Hour")==h)
    point_forecasts = df_hour.select(pl.col("Naive")).to_numpy().ravel()
    obs = df_hour.select(pl.col("Price")).to_numpy().ravel()

    for t in range(window+1, len(df)//24):
        lower, upper = method(point_forecasts[t-window:t], obs[t-window:t], alpha)
        quantile_forecasts[t, h, 0] = lower + point_forecasts[t]
        quantile_forecasts[t, h, 1] = upper + point_forecasts[t]

In [73]:
df = df.select(pl.col("Date"), pl.col("Hour"), pl.col("Price"), pl.col("Naive"),)
df = df.with_columns(LowerQuantile = pl.Series(quantile_forecasts[:, :, 0].flatten()))  
df = df.with_columns(UpperQuantile = pl.Series(quantile_forecasts[:, :, 1].flatten()))  
# note that flatten() by default applies the row-major order, you can read more details at https://numpy.org/doc/2.3/reference/generated/numpy.ndarray.flatten.html
# our forecasts are in the (days, hours) shape, so we want the row-major order
# if we stored our forecasts in the (hours, day) shape, then we would have to call `flatten(order='F')` to apply the column-major order
df = df.drop_nans()
df

Date,Hour,Price,Naive,LowerQuantile,UpperQuantile
f64,u8,f64,f64,f64,f64
2.0120108e7,0,27.97,30.43,19.847,41.013
2.0120108e7,1,27.62,29.275,20.837,37.713
2.0120108e7,2,23.56,26.785,18.615,34.955
2.0120108e7,3,23.08,26.24,17.9315,34.5485
2.0120108e7,4,23.18,26.04,18.08,34.0
…,…,…,…,…,…
2.0131217e7,19,113.92,100.745,61.631,139.859
2.0131217e7,20,107.26,96.265,65.102,127.428
2.0131217e7,21,89.02,82.08,54.481,109.679


In [74]:
def pinball(y, q, level):
    return (y-q)*(level - (y < q))

def hits(y, q):
    return y < q


In [75]:
df.select(
    pinball(pl.col("Price"), pl.col("LowerQuantile"), (1-alpha)/2).mean().alias("PinbalLowerQuantile"),
    pinball(pl.col("Price"), pl.col("UpperQuantile"),  (1-(1-alpha)/2)).mean().alias("PinbalUpperQuantile"),
    hits(pl.col("Price"), pl.col("LowerQuantile")).mean().alias("CoverageLowerQuantile"),
    hits(pl.col("Price"), pl.col("UpperQuantile")).mean().alias("CoverageUpperQuantile"),
)

PinbalLowerQuantile,PinbalUpperQuantile,CoverageLowerQuantile,CoverageUpperQuantile
f64,f64,f64,f64
1.779545,2.275551,0.055164,0.934683
